In [2]:
!pip3 install yfinance
!pip3 install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=57b7bc06fc148907bdf4d180bd3a411fe45d1a27f9837cb501d8df66a2eb7cf0
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import os
import matplotlib.pyplot as plt
data = yf.download(tickers = 'ATVI', start = '2008-03-11',end = '2023-07-10')
data.head(5)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-03-11,13.545,13.615,13.365,13.565,11.898015,8085400
2008-03-12,13.685,13.685,13.380,13.410,11.762063,7444400
2008-03-13,13.375,13.650,13.255,13.615,11.941870,7062200
2008-03-14,13.670,13.690,13.365,13.470,11.814688,8038000
2008-03-17,13.405,13.535,13.125,13.270,11.639267,9579400


In [4]:
data['Tomorrow'] = data["Close"].shift(-1)
data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)
data.dropna(inplace=True)
data.reset_index(inplace = True)

In [5]:
data_set = data.iloc[:, 0:11]#.values
pd.set_option('display.max_columns', None)

data_set.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Tomorrow,Target
0,2008-03-11,13.545,13.615,13.365,13.565,11.898015,8085400,13.410,0
1,2008-03-12,13.685,13.685,13.380,13.410,11.762063,7444400,13.615,1
2,2008-03-13,13.375,13.650,13.255,13.615,11.941870,7062200,13.470,0
3,2008-03-14,13.670,13.690,13.365,13.470,11.814688,8038000,13.270,0
4,2008-03-17,13.405,13.535,13.125,13.270,11.639267,9579400,13.555,1


In [6]:
def make_predictions(training_data, testing_data, features, ml_model):
    ml_model.fit(training_data[features], training_data["Target"])
    predictions = ml_model.predict_proba(testing_data[features])[:,1]
    predictions[predictions >= 0.55] = 1
    predictions[predictions < 0.55] = 0
    predictions = pd.Series(predictions, index=testing_data.index, name="Predictions")
    combined_result = pd.concat([testing_data["Target"], predictions], axis=1)
    return combined_result

In [7]:
def custom_backtest(data, machine_learning_model, features_to_use, start_index=2000, step_size=250):
    all_predictions = []

    for i in range(start_index, data.shape[0], step_size):
        training_data = data.iloc[0:i].copy()
        testing_data = data.iloc[i:(i+step_size)].copy()
        predictions = make_predictions(training_data, testing_data, features_to_use, machine_learning_model)
        all_predictions.append(predictions)

    return pd.concat(all_predictions)

In [ ]:
horizons = [2,5,60,250,1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = data_set.rolling(horizon).mean()

    ratio_column = f"Close_Ratio_{horizon}"
    data_set[ratio_column] = data_set["Close"] / rolling_averages["Close"]

    trend_column = f"Trend_{horizon}"
    data_set[trend_column] = data_set.shift(1).rolling(horizon).sum()["Target"]

    new_predictors += [ratio_column, trend_column]

In [9]:
data_set = data_set.dropna(subset=data_set.columns[data_set.columns != "Tomorrow"])

In [11]:
import csv
with open('checked_news.csv', mode='r') as csv_file:
    csv_reader = csv.reader(csv_file)
    data = [row for row in csv_reader]

    for row in data:
        del row[1]
    news = dict()
    date = None
    indexes = {'?': 0, '-': 0, '+': 0}
    for row in data:
        if row[0] == 'date':
            continue
        prev_date = date
        date = row[0]
        if prev_date != date and prev_date != None:
            if indexes['-'] > indexes['+']:
                news[prev_date] = -1
            else:
                news[prev_date] = +1
            indexes = {'?': 0, '-': 0, '+': 0}
        if row[0] not in news:
            news[row[0]] = row[1]
            indexes[row[1]] += 1
        else:
            indexes[row[1]] += 1

new_data = {'Date': [],
            'News': []}
for i in news:
    new_data['Date'].append(i)
    new_data['News'].append(news[i])

new_data = pd.DataFrame(new_data)


data_set['Date'] = data_set['Date'].astype(str)
new_data['Date'] = new_data['Date'].astype(str)
merged_df = pd.merge(data_set, new_data, on='Date', how='left')
merged_df.fillna(0, inplace=True)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

model = RandomForestClassifier(n_estimators=2000, min_samples_split=500, random_state=1)

train = merged_df.iloc[:-800]
test = merged_df.iloc[-800:]

predictors = ["Adj Close", "Open", "High", "Low", "Close", "Volume", "News"]
model.fit(train[predictors], train["Target"])

RandomForestClassifier(min_samples_split=500, n_estimators=2000, random_state=1)

In [13]:
predictions = custom_backtest(merged_df, model, new_predictors)

In [14]:
predictions["Predictions"].value_counts()

0.0    712
1.0    145
Name: Predictions, dtype: int64

In [15]:
precision_score(predictions["Target"], predictions["Predictions"])

0.5655172413793104

In [16]:
predictions

,Target,Predictions
2000,0,0.0
2001,1,0.0
2002,0,0.0
2003,1,0.0
2004,1,0.0
...,...,...
2852,1,0.0
2853,0,0.0
2854,0,0.0
2855,0,0.0
